In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

In [ ]:
index = 291951
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
dane_testy = pd.read_csv('https://gist.githubusercontent.com/sikor272/a0ddd825ea715b35518215087090b70a/raw/d0f298961aabf5d6e2635b803b57ef24827d3265/dane_testy.csv', sep=';')

In [ ]:
columns = ['Lokalizacja', 'Typ', 'Metoda', 'Liczba_uczniów', 'Płeć', 'Obiad', 'Pre_test']

In [ ]:
for i in columns:
  print(i, dane_testy[i].unique())

In [ ]:
print('Szkoła', dane_testy['Szkoła'].unique(), len(dane_testy['Szkoła'].unique()))
print('Klasa', dane_testy['Klasa'].unique(), len(dane_testy['Klasa'].unique()))
print('Uczeń', dane_testy['Uczeń'].unique(), len(dane_testy['Uczeń'].unique()))

In [ ]:
dane_testy['Lokalizacja_Miejska'] = dane_testy['Lokalizacja'].apply(lambda s: 1 if s == 'miejska' else 0)
dane_testy['Lokalizacja_Podmiejska'] = dane_testy['Lokalizacja'].apply(lambda s: 1 if s == 'podmiejska' else 0)
dane_testy['Lokalizacja_Wiejska'] = dane_testy['Lokalizacja'].apply(lambda s: 1 if s == 'wiejska' else 0)
dane_testy['Typ'] = dane_testy['Typ'].apply(lambda s: 1 if s == 'publiczna' else 0)
dane_testy['Metoda'] = dane_testy['Metoda'].apply(lambda s: 1 if s == 'standardowa' else 0)
dane_testy['Płeć'] = dane_testy['Płeć'].apply(lambda s: 1 if s == 'chłopiec' else 0)
dane_testy['Obiad'] = dane_testy['Obiad'].apply(lambda s: 1 if s == 'kwalifikuje się' else 0)

In [ ]:
columns = ['Lokalizacja_Miejska', 'Lokalizacja_Podmiejska', 'Lokalizacja_Wiejska', 'Typ', 'Metoda', 'Liczba_uczniów', 'Płeć', 'Obiad', 'Pre_test']

**Macierz korelacji**

In [ ]:
sns.heatmap(dane_testy[columns + ['Post_test']].corr(), annot=True, fmt='.2f', annot_kws={'size': 15}, yticklabels=columns + ['Post_test'], xticklabels=columns + ['Post_test'])
plt.show()

**Macierz wykresów rozrzutów**

In [ ]:
sns.pairplot(dane_testy, x_vars=columns, y_vars=["Post_test"], diag_kind = None)
plt.tight_layout()
plt.show()

**Podział danych**

In [ ]:
columns_after = ['Metoda', 'Pre_test']

In [ ]:
x_train_all, x_test_all , y_train_all, y_test_all = model_selection.train_test_split(dane_testy[columns], dane_testy['Post_test'], train_size=0.7, random_state=index)

x_train, x_test , y_train, y_test = model_selection.train_test_split(dane_testy[columns_after], dane_testy['Post_test'], train_size=0.7, random_state=index)
y_train = np.array(y_train).reshape((-1, 1))
y_test = np.array(y_test).reshape((-1, 1))

In [ ]:
print(dane_testy.shape)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

**Sieć neuronowa**


In [ ]:
scaler_x = MinMaxScaler()
scaler_x.fit(x_train)
x_train_norm = scaler_x.transform(x_train)
x_test_norm = scaler_x.transform(x_test)

scaler_y = MinMaxScaler()
scaler_y.fit(y_train)
y_train_norm = scaler_y.transform(y_train)
y_test_norm = scaler_y.transform(y_test)

In [ ]:
siec_neur = MLPRegressor(hidden_layer_sizes=(10,), activation='tanh',solver='lbfgs', alpha=0.0001, max_iter = 10000, random_state=index)
_ = siec_neur.fit(x_train_norm, y_train_norm.ravel())

In [ ]:
siec_neur.coefs_

In [ ]:
y_predict_train = siec_neur.predict(x_train_norm)
y_predict_train = y_predict_train.reshape((-1, 1))
y_predict_train_denorm = scaler_y.inverse_transform(y_predict_train)

y_predict_test = siec_neur.predict(x_test_norm)
y_predict_test = y_predict_test.reshape((-1, 1))
y_predict_test_denorm = scaler_y.inverse_transform(y_predict_test)

In [ ]:
print('MSE dla uczacej ', mean_squared_error(y_train, y_predict_train_denorm))
print('MAE dla uczacej  ', mean_absolute_error(y_train, y_predict_train_denorm))
print('RMSE dla uczacej ', np.sqrt(mean_squared_error(y_train, y_predict_train_denorm)))
print('')
print('MSE dla testowej', mean_squared_error(y_test, y_predict_test_denorm))
print('MAE dla testowej ', mean_absolute_error(y_test, y_predict_test_denorm))
print('RMSE dla testowej', np.sqrt(mean_squared_error(y_test, y_predict_test_denorm)))

In [ ]:
plt.scatter(x=y_test, y=y_predict_test_denorm)
plt.xlabel('Wartości rzeczywiste')
plt.ylabel('Wartości przewidywane')
plt.show()

**Drzewo CART** wszystkie predyktory

In [ ]:
drzewo = DecisionTreeRegressor(criterion="mse", max_depth=5, min_samples_split=10, min_samples_leaf=40, random_state=index)
_ = drzewo.fit(x_train_all, y_train_all)

In [ ]:
for i, j in zip(columns, drzewo.feature_importances_):
  print(i, ' ważność:', j)

**Drzewo CART**

In [ ]:
drzewo = DecisionTreeRegressor(criterion="mse", max_depth=5, min_samples_split=10, min_samples_leaf=40, random_state=index)
_ = drzewo.fit(x_train, y_train)

In [ ]:
for i, j in zip(columns_after, drzewo.feature_importances_):
  print(i, ' ważność:', j)

In [ ]:
graphviz.Source(export_graphviz(drzewo, filled=True, feature_names=columns_after))

In [ ]:
y_predict_train_cart = drzewo.predict(x_train)
y_predict_test_cart = drzewo.predict(x_test)

In [ ]:
print('MSE dla uczacej ', mean_squared_error(y_train, y_predict_train_cart))
print('MAE dla uczacej  ', mean_absolute_error(y_train, y_predict_train_cart))
print('RMSE dla uczacej ', np.sqrt(mean_squared_error(y_train, y_predict_train_cart)))
print('')
print('MSE dla testowej', mean_squared_error(y_test, y_predict_test_cart))
print('MAE dla testowej ', mean_absolute_error(y_test, y_predict_test_cart))
print('RMSE dla testowej', np.sqrt(mean_squared_error(y_test, y_predict_test_cart)))

In [ ]:
plt.scatter(x=y_test, y=y_predict_test_cart)
plt.xlabel('Wartości rzeczywiste')
plt.ylabel('Wartości przewidywane')
plt.show()